In [1]:
######################
### INITIALIZATION ###
######################
# mount Google Drive and switch to zatoboj folder in Files view
from google.colab import drive
drive.mount('/content/gdrive')  
from google.colab import files
files.view('/content/gdrive/MyDrive/PUSHMI/pushmi_pullyu') 
# if you want to work with git from this notebook
%cd /content/gdrive/My\ Drive/PUSHMI/pushmi_pullyu
!mkdir -p ~/.ssh
!cp ../id_rsa ~/.ssh
!ssh-keyscan -t rsa github.com >> /root/.ssh/known_hosts
!ssh -T git@github.com
!source ../git_config_startup.sh
# install module luster
!pip install -q -q -e .
# import module ztpt
import pushmi
# set up module autoreload
%load_ext autoreload
%autoreload 2
# check your GPU
!nvidia-smi
# some useful imports for training
import wandb
from pushmi.conf import default_config
from pushmi.model import get_model_name

Mounted at /content/gdrive


<IPython.core.display.Javascript object>

/content/gdrive/My Drive/PUSHMI/pushmi_pullyu
# github.com:22 SSH-2.0-babeld-9d6fcaa2
Hi eduryev! You've successfully authenticated, but GitHub does not provide shell access.
     |████████████████████████████████| 686kB 13.8MB/s 
     |████████████████████████████████| 1.9MB 32.0MB/s 
     |████████████████████████████████| 102kB 15.5MB/s 
     |████████████████████████████████| 829kB 60.7MB/s 
     |████████████████████████████████| 276kB 56.8MB/s 
     |████████████████████████████████| 163kB 57.7MB/s 
     |████████████████████████████████| 102kB 13.9MB/s 
     |████████████████████████████████| 102kB 14.5MB/s 
     |████████████████████████████████| 133kB 52.2MB/s 
     |████████████████████████████████| 1.3MB 50.4MB/s 
     |████████████████████████████████| 71kB 8.7MB/s 
     |████████████████████████████████| 143kB 56.7MB/s 
     |████████████████████████████████| 296kB 58.9MB/s 
Tue Jan 12 15:04:52 2021       
+------------------------------------------------------------------

# Git cells

In [ ]:
!git fetch
!git status

In [ ]:
!git add .
!git commit -m 'conf set up'

In [ ]:
!git push

# Training one model

In [ ]:
config = default_config()
root_dir = config.dirs.root

config.model.batch_size = 100   
config.model.lr = 1e-4
config.model.loss_type = 'distance'
config.model.signature = 'kompot-1'
config.train.max_epochs = 200
   
name = get_model_name(config)
wandb.init(name=name, project='pushmi', dir=root_dir, reinit=True);
model = pushmi.train_model(config)

# Training many models

In [ ]:
import wandb
import itertools
from pushmi.conf import default_config
from pushmi.model import get_model_name

config = default_config()
root_dir = config.dirs.root

max_epochs = 150
batch_sizes = [300, 500, 100]
lrs = [1e-4, 1e-5, 5e-5, 1e-6]
loss_types = ['distance', 'entropy']

for loss_type, batch_size, lr in itertools.product(loss_types, batch_sizes, lrs):
    config = default_config()
    config.model.batch_size = batch_size   
    config.model.lr = lr
    config.model.loss_type = loss_type
    config.model.signature = 'belka-1'
    config.train.max_epochs = max_epochs
    name = get_model_name(config)
    wandb.init(name=name, project='pushmi', dir=root_dir, reinit=True);
    model = pushmi.train_model(config)

# Antonov treshak

In [ ]:
model1 = models[0]
d0 = model1.full_eval(mode='train')
predicted_labels = d0['predicted_labels']
true_labels = d0['true_labels']
pushmi.get_cluster_acc(true_labels, predicted_labels)

In [ ]:
individual_accs, predictions, true_labels, final_predictions = pushmi.combine_predictions(models)

In [ ]:
from pprint import pprint
from collections import Counter
for pred in predictions:
    pprint(pred[final_predictions != true_labels])
pprint(true_labels[final_predictions != true_labels])
print(Counter(true_labels[final_predictions != true_labels]))
print(Counter(true_labels))

array([0, 8, 7, ..., 5, 7, 2])
array([4, 2, 4, ..., 6, 4, 7])
Counter({6: 510, 3: 489, 4: 447, 8: 369, 5: 364, 9: 353, 2: 280, 0: 209, 7: 142, 1: 46})
Counter({1: 567, 8: 513, 7: 511, 6: 510, 2: 506, 9: 504, 3: 489, 0: 488, 5: 465, 4: 447})


In [ ]:
counters = [Counter([pred[i] for pred in predictions]) for i in range(true_labels.shape[0])]

In [ ]:
final_predictions = np.array([c.most_common(1)[0][0] for c in counters])

In [ ]:
sum(final_predictions[:] != true_labels[:])

233

In [ ]:
4767/5000

0.9534

[]

In [ ]:
i = -1
predicted_labels = val_results[i]['predicted_labels'].cpu().detach().numpy()
true_labels = val_results[i]['true_labels'].cpu().detach().numpy()
acc = val_results[i]['acc']
accs = [x['acc'] for x in val_results]

In [ ]:
d1['true_labels'] - d0['true_labels']

array([0, 0, 0, ..., 0, 0, 0])

In [ ]:
d1['true_labels'] - val_results[-1]['true_labels'].cpu().detach().numpy()

array([ 0, -2,  4, ...,  4, -4, -2])

In [ ]:
Counter(d0['true_labels']), Counter(val_results[-1]['true_labels'].cpu().detach().numpy())

(Counter({0: 488,
          1: 567,
          2: 506,
          3: 489,
          4: 447,
          5: 465,
          6: 510,
          7: 511,
          8: 513,
          9: 504}),
 Counter({0: 488,
          1: 567,
          2: 506,
          3: 489,
          4: 447,
          5: 465,
          6: 510,
          7: 511,
          8: 513,
          9: 504}))

In [ ]:
Counter(d1['true_labels']), Counter(val_results[-1]['true_labels'].cpu().detach().numpy())

(Counter({0: 466,
          1: 539,
          2: 478,
          3: 505,
          4: 514,
          5: 468,
          6: 499,
          7: 535,
          8: 493,
          9: 503}),
 Counter({0: 488,
          1: 567,
          2: 506,
          3: 489,
          4: 447,
          5: 465,
          6: 510,
          7: 511,
          8: 513,
          9: 504}))

In [ ]:
d1['true_labels'].shape, true_labels.shape

((5000,), (5000,))

In [ ]:
accs

[0.245, 0.6774, 0.335, 0.6458]

In [ ]:
predicted_labels = predicted_labels.cpu().detach().numpy()
true_labels = true_labels.cpu().detach().numpy()

In [ ]:
len(true_labels), len(predicted_labels)

(5000, 5000)

In [ ]:
from collections import Counter
from pprint import pprint
for i in range(10):
    #i = 2
    print(i, Counter(predicted_labels[true_labels==i]), Counter(true_labels[predicted_labels==i]))

0 Counter({6: 465, 7: 1}) Counter({6: 495, 4: 2, 5: 2, 8: 1})
1 Counter({2: 530, 8: 6, 5: 2, 3: 1}) Counter({9: 490, 4: 12, 8: 7, 7: 6, 3: 3, 5: 2, 2: 1})
2 Counter({8: 467, 5: 4, 3: 3, 4: 2, 6: 1, 1: 1}) Counter({1: 530, 4: 3, 8: 1, 9: 1, 6: 1})
3 Counter({3: 498, 1: 3, 5: 2, 7: 1, 4: 1}) Counter({3: 498, 9: 10, 8: 4, 2: 3, 1: 1})
4 Counter({9: 496, 1: 12, 2: 3, 0: 2, 4: 1}) Counter({7: 522, 2: 2, 4: 1, 3: 1})
5 Counter({7: 462, 1: 2, 0: 2, 5: 1, 6: 1}) Counter({8: 475, 2: 4, 9: 2, 1: 2, 3: 2, 6: 2, 5: 1})
6 Counter({0: 495, 5: 2, 9: 1, 2: 1}) Counter({0: 465, 2: 1, 5: 1, 8: 1})
7 Counter({4: 522, 1: 6, 8: 4, 9: 3}) Counter({5: 462, 0: 1, 3: 1})
8 Counter({5: 475, 1: 7, 3: 4, 9: 3, 8: 1, 2: 1, 0: 1, 6: 1}) Counter({2: 467, 1: 6, 7: 4, 8: 1})
9 Counter({1: 490, 3: 10, 5: 2, 2: 1}) Counter({4: 496, 7: 3, 8: 3, 6: 1})


In [ ]:
from collections import Counter
from pprint import pprint
counters = []
for i in range(10):
    #i = 2
    counters.append(Counter(predicted_labels[true_labels==i]))
    print('Breakdown by new classes for true label', i, ': ', Counter(predicted_labels[true_labels==i]))#, Counter(true_labels[predicted_labels==i]))

Breakdown by new classes for true label 0 :  Counter({6: 465, 7: 1})
Breakdown by new classes for true label 1 :  Counter({2: 530, 8: 6, 5: 2, 3: 1})
Breakdown by new classes for true label 2 :  Counter({8: 467, 5: 4, 3: 3, 4: 2, 6: 1, 1: 1})
Breakdown by new classes for true label 3 :  Counter({3: 498, 1: 3, 5: 2, 7: 1, 4: 1})
Breakdown by new classes for true label 4 :  Counter({9: 496, 1: 12, 2: 3, 0: 2, 4: 1})
Breakdown by new classes for true label 5 :  Counter({7: 462, 1: 2, 0: 2, 5: 1, 6: 1})
Breakdown by new classes for true label 6 :  Counter({0: 495, 5: 2, 9: 1, 2: 1})
Breakdown by new classes for true label 7 :  Counter({4: 522, 1: 6, 8: 4, 9: 3})
Breakdown by new classes for true label 8 :  Counter({5: 475, 1: 7, 3: 4, 9: 3, 8: 1, 2: 1, 0: 1, 6: 1})
Breakdown by new classes for true label 9 :  Counter({1: 490, 3: 10, 5: 2, 2: 1})


In [ ]:
sum([max(counter.values()) for counter in counters]) / sum([sum(counter.values()) for counter in counters]), acc

(0.98, 0.98)

In [ ]:
from collections import Counter
from pprint import pprint
counters = []
for i in range(10):
    #i = 2
    counters.append(Counter(predicted_labels[true_labels==i]))
    print('Breakdown by new classes for true label', i, ': ', Counter(predicted_labels[true_labels==i]))#, Counter(true_labels[predicted_labels==i]))

Breakdown by new classes for true label 0 :  Counter({2: 457, 4: 4, 9: 1})
Breakdown by new classes for true label 1 :  Counter({0: 556, 5: 11, 3: 5, 1: 4, 9: 3})
Breakdown by new classes for true label 2 :  Counter({5: 485, 1: 5, 8: 1, 3: 1, 6: 1})
Breakdown by new classes for true label 3 :  Counter({6: 504, 1: 10, 7: 10, 5: 1})
Breakdown by new classes for true label 4 :  Counter({3: 473, 8: 2, 0: 1, 5: 1, 4: 1})
Breakdown by new classes for true label 5 :  Counter({7: 455, 4: 1, 9: 1, 0: 1, 6: 1})
Breakdown by new classes for true label 6 :  Counter({4: 489})
Breakdown by new classes for true label 7 :  Counter({1: 537, 8: 2, 5: 2, 7: 1, 3: 1})
Breakdown by new classes for true label 8 :  Counter({9: 480, 0: 5, 8: 3, 4: 2, 7: 1, 3: 1, 6: 1})
Breakdown by new classes for true label 9 :  Counter({8: 449, 1: 11, 7: 9, 3: 5, 2: 2, 9: 2, 0: 1})


In [ ]:
Counter(predicted_labels), Counter(true_labels)

(Counter({0: 471,
          1: 495,
          2: 488,
          3: 550,
          4: 515,
          5: 513,
          6: 507,
          7: 447,
          8: 446,
          9: 568}),
 Counter({0: 504,
          1: 569,
          2: 480,
          3: 564,
          4: 495,
          5: 451,
          6: 489,
          7: 508,
          8: 477,
          9: 463}))

In [ ]:
logits =  val_results[-1]['logits']

In [ ]:
logits[5:15]

tensor([[0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
         0.0000e+00, 0.0000e+00, 0.0000e+00, 1.0000e+00],
        [9.7421e-40, 1.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
         0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00],
        [0.0000e+00, 0.0000e+00, 1.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
         0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00],
        [0.0000e+00, 0.0000e+00, 1.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
         0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00],
        [0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
         0.0000e+00, 1.0000e+00, 0.0000e+00, 0.0000e+00],
        [0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
         0.0000e+00, 0.0000e+00, 0.0000e+00, 1.0000e+00],
        [0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 1.0000e+00, 0.0000e+00,
         0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00],
        [0.0000e+00, 0.0000

In [ ]:
true_labels[5:15]

array([1, 8, 0, 0, 7, 1, 2, 7, 0, 5])

In [ ]:
model.entropy_loss(torch.mean(logits[:5], axis=0))

tensor(-2.3026, device='cuda:0')